<a href="https://colab.research.google.com/github/inlima/kaggle_titanic_competition/blob/main/Titanic_case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Case Titanic

## Importando Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pylab 
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

## Importando Arquivos

In [ ]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

## Estudando Variáveis

In [ ]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
train = train.drop(columns = ['Cabin'])
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 76.7+ KB


In [ ]:
le = LabelEncoder()
obj = [column for column in train.columns if train[column].dtype == 'object']
for column in obj:
    train[column] = le.fit_transform(train[column])

train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Survived
0,1,3,108,1,22.0,1,0,523,7.2500,2,0
1,2,1,190,0,38.0,1,0,596,71.2833,0,1
2,3,3,353,0,26.0,0,0,669,7.9250,2,1
3,4,1,272,0,35.0,1,0,49,53.1000,2,1
4,5,3,15,1,35.0,0,0,472,8.0500,2,0


In [ ]:
train['Age'] = train['Age'].fillna(train['Age'].mean())
train['Age'].describe() 

count    891.000000
mean      29.699118
std       13.002015
min        0.420000
25%       22.000000
50%       29.699118
75%       35.000000
max       80.000000
Name: Age, dtype: float64

In [ ]:
train_age = train.sort_values(by=['Age'])
train_age.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
803,804,1,3,807,1,0.42,0,1,174,8.5167,0
755,756,1,2,321,1,0.67,1,1,166,14.5000,2
644,645,1,3,53,0,0.75,2,1,194,19.2583,0
469,470,1,3,54,0,0.75,2,1,194,19.2583,0
831,832,1,2,686,1,0.83,1,1,237,18.7500,2
78,79,1,2,127,1,0.83,0,2,158,29.0000,2
305,306,1,1,16,1,0.92,1,2,37,151.5500,2
164,165,0,3,628,1,1.00,4,1,249,39.6875,2
827,828,1,2,503,1,1.00,0,2,618,37.0042,0
172,173,1,3,408,0,1.00,1,1,344,11.1333,2


In [ ]:
train = train.reindex(columns = ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
                                    'Parch', 'Ticket', 'Fare', 'Embarked','Survived'])  

In [ ]:
train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Survived
0,1,3,108,1,22.0,1,0,523,7.2500,2,0
1,2,1,190,0,38.0,1,0,596,71.2833,0,1
2,3,3,353,0,26.0,0,0,669,7.9250,2,1
3,4,1,272,0,35.0,1,0,49,53.1000,2,1
4,5,3,15,1,35.0,0,0,472,8.0500,2,0


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [ ]:
test = test.drop(columns = ['Cabin'])

In [ ]:
obj = [column for column in test.columns if test[column].dtype == 'object']
for column in obj:
    test[column] = le.fit_transform(test[column])

In [ ]:
test['Age'] = test['Age'].fillna(test['Age'].mean())
test['Age'].describe()

count    418.000000
mean      30.272590
std       12.634534
min        0.170000
25%       23.000000
50%       30.272590
75%       35.750000
max       76.000000
Name: Age, dtype: float64

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    int64  
 3   Sex          418 non-null    int64  
 4   Age          418 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    int64  
 8   Fare         417 non-null    float64
 9   Embarked     418 non-null    int64  
dtypes: float64(2), int64(8)
memory usage: 32.8 KB


In [ ]:
test['Fare'] = test['Fare'].fillna(test['Fare'].mean())

## Implementando o Modelo

In [ ]:
X = train[['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
              'Parch', 'Ticket', 'Fare', 'Embarked']]
y = train['Survived']

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X,y)

KNeighborsClassifier()

In [ ]:
scores = knn.predict(test)
scores

array([0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0,

In [ ]:
cv = RepeatedKFold(n_splits=10,n_repeats=5)

acc = cross_val_score(knn,X,y,cv=cv,scoring='accuracy')
acc.mean()

0.6359300873907615

In [ ]:
k_range = list(range(1, 31))
param_grid = dict(n_neighbors=k_range)

grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy', return_train_score=False,verbose=1)

In [ ]:
grid_search=grid.fit(X,y)

Fitting 10 folds for each of 30 candidates, totalling 300 fits


In [ ]:
print(grid_search.best_params_)

{'n_neighbors': 18}


In [ ]:
print(grid_search.best_score_ *100)

66.44319600499375


In [ ]:
knn = KNeighborsClassifier(n_neighbors=18)
knn.fit(X,y)

scores = knn.predict(test)

cv = RepeatedKFold(n_splits=10,n_repeats=5)

acc = cross_val_score(knn,X,y,cv=cv,scoring='accuracy')
acc.mean()

0.6549987515605493

## Gerando o arquivo de submissão

In [ ]:
sub = test[['PassengerId']]
sub['Survived'] = scores
sub

In [ ]:
sub.to_csv("submission.csv", index=False)